In [1]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline

In [3]:
cnxn_string = ("postgresql+psycopg2://{username}:{pswd}"
              "@{host}:{port}/{database}")
print(cnxn_string)

postgresql+psycopg2://{username}:{pswd}@{host}:{port}/{database}


In [4]:
engine = create_engine(cnxn_string.format(
    username="postgres",
    pswd="sciBow1237",
    host="localhost",
    port=5432,
    database="sqlda"))

#SQLAlchemy 2.0 syntax
with engine.begin() as connection:
    connection.execute(text("SELECT * FROM movies LIMIT 2;")).fetchall()

[(1, 4, 'Iron Man', datetime.date(2008, 5, 2), 126, 1, 1),
 (2, 6, 'The Incredible Hulk', datetime.date(2008, 6, 13), 112, 1, 1)]

In [5]:
mps_data = pd.read_sql_table('movies_plus_soundtracks',engine)
mps_data

,chron_num,timeline_num,title,release_date,runtime_min,phase_id,saga_id,first_name,last_name,track_count,duration_min,downloaded_tracks,downloaded_duration
0,1,4,Iron Man,2008-05-02,126,1,1,Ramin,Djawadi,19,54,12,34
1,2,6,The Incredible Hulk,2008-06-13,112,1,1,Craig,Armstrong,45,111,21,62
2,3,5,Iron Man 2,2010-05-07,125,1,1,John,Debney,25,72,16,56
3,4,7,Thor,2011-05-06,114,1,1,Patrick,Doyle,24,72,12,41
4,5,1,Captain America: The First Avenger,2011-07-22,124,1,1,Alan,Silvestri,27,74,15,42
5,6,8,The Avengers,2012-05-04,143,1,1,Alan,Silvestri,18,65,12,45
6,7,10,Iron Man 3,2013-05-03,131,2,1,Brian,Tyler,20,76,11,47
7,8,9,Thor: The Dark World,2013-11-08,112,2,1,Brian,Tyler,26,77,17,52
8,9,11,Captain America: The Winter Soldier,2014-04-04,136,2,1,Henry,Jackman,20,71,10,41
9,10,12,Guardians of the Galaxy,2014-08-01,122,2,1,Tyler,Bates,29,65,16,48
